# APIs & Authentication

In [2]:
import requests
import pandas as pd
import json as js

## API Key
**OpenWeatherMap → API Key** (simple key in query string) <br>
Get keys from https://home.openweathermap.org/api_keys

In [ ]:
## New API keys can take up to 1–2 hours to become active.
API_KEY = ""  # put your real key here
city = "Kent"
# f"" → lets you insert variables/expressions inside a string.
url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"

response = requests.get(url)

# Check if the request worked
if response.status_code == 200:
    data = response.json()
    print("City:", data["name"])
    print("Weather:", data["weather"][0]["description"])
    print("Temperature (C):", data["main"]["temp"])
else:
    print("Error:", response.status_code, response.text)

City: Kent
Weather: overcast clouds
Temperature (C): 18.39


## Create a function 
Create a function so we can re-use the code and tidy up our code

In [12]:
def displayResponse(response):
    if response.status_code == 200:
        data = response.json()
        print(js.dumps(data, indent=2))
    else:
        print("Error:", response.status_code, response.text)

### Use this function

In [13]:
displayResponse(response)

{
  "coord": {
    "lon": 0.6667,
    "lat": 51.1667
  },
  "weather": [
    {
      "id": 804,
      "main": "Clouds",
      "description": "overcast clouds",
      "icon": "04d"
    }
  ],
  "base": "stations",
  "main": {
    "temp": 18.39,
    "feels_like": 18.82,
    "temp_min": 18.26,
    "temp_max": 19.54,
    "pressure": 1010,
    "humidity": 97,
    "sea_level": 1010,
    "grnd_level": 1003
  },
  "visibility": 10000,
  "wind": {
    "speed": 1.34,
    "deg": 196,
    "gust": 4.47
  },
  "clouds": {
    "all": 100
  },
  "dt": 1758366672,
  "sys": {
    "type": 2,
    "id": 19047,
    "country": "GB",
    "sunrise": 1758346824,
    "sunset": 1758391273
  },
  "timezone": 3600,
  "id": 3333158,
  "name": "Kent",
  "cod": 200
}


## Personal Access Token
Now, we will look at github's personal access token (PAT)
- Create a **Personal Access Token (fine-grained or classic)** from your GitHub settings -> Developer Settings -> Fine-grained tokens.

In [ ]:
GITHUB_TOKEN = ""

gh_headers = {
    "Authorization": f"Bearer {GITHUB_TOKEN}",
    "Accept": "application/vnd.github+json",
    # (optional but recommended)
    "X-GitHub-Api-Version": "2022-11-28",
    "User-Agent": "api-auth-demo"
}

user = requests.get("https://api.github.com/user", headers=gh_headers, timeout=30)
displayResponse(user)

{
  "login": "moresandeep",
  "id": 380997,
  "node_id": "MDQ6VXNlcjM4MDk5Nw==",
  "avatar_url": "https://avatars.githubusercontent.com/u/380997?v=4",
  "gravatar_id": "",
  "url": "https://api.github.com/users/moresandeep",
  "html_url": "https://github.com/moresandeep",
  "followers_url": "https://api.github.com/users/moresandeep/followers",
  "following_url": "https://api.github.com/users/moresandeep/following{/other_user}",
  "gists_url": "https://api.github.com/users/moresandeep/gists{/gist_id}",
  "starred_url": "https://api.github.com/users/moresandeep/starred{/owner}{/repo}",
  "subscriptions_url": "https://api.github.com/users/moresandeep/subscriptions",
  "organizations_url": "https://api.github.com/users/moresandeep/orgs",
  "repos_url": "https://api.github.com/users/moresandeep/repos",
  "events_url": "https://api.github.com/users/moresandeep/events{/privacy}",
  "received_events_url": "https://api.github.com/users/moresandeep/received_events",
  "type": "User",
  "user_vie

### List my public repos (first 10)

In [16]:
repos = requests.get("https://api.github.com/user/repos", headers=gh_headers, params={"per_page": 10}, timeout=30)
reposJson = repos.json()
rows = [{
    "Name": r.get("name"),
    "Private": r.get("private"),
    "Language": r.get("language"),
    "Stars": r.get("stargazers_count"),
    "URL": r.get("html_url")
} for r in reposJson]

pd.DataFrame(rows)

,Name,Private,Language,Stars,URL
0,apache-boxer-test,False,None,0,https://github.com/apache/apache-boxer-test
1,apache-website-template,False,None,36,https://github.com/apache/apache-website-template
2,knox,False,Java,205,https://github.com/apache/knox
3,orc,False,Java,743,https://github.com/apache/orc
4,orc-format,False,None,13,https://github.com/apache/orc-format
5,experience-primer-copilot-moresandeep,False,None,0,https://github.com/Exp-Primer-Copilot-Cohort-1...
6,ansible-hortonworks,False,Python,248,https://github.com/hortonworks/ansible-hortonw...
7,bigdata-interop-fork,False,Java,0,https://github.com/hortonworks/bigdata-interop...
8,bman,False,Python,3,https://github.com/hortonworks/bman
9,cb-cli,False,Go,12,https://github.com/hortonworks/cb-cli


## App Password
Bluesky — App Passwords (Safer than your main password)
- In the Bluesky app: **Settings → Privacy and Security → App passwords**. Create one.
- **Do NOT** use your main password here.
- Flow:
  1) `POST https://bsky.social/xrpc/com.atproto.server.createSession` with `identifier` (your handle) + `password` (app password)
  2) Receive `accessJwt` (short-lived)
  3) Call an endpoint like `GET /xrpc/app.bsky.feed.getTimeline` with `Authorization: Bearer <accessJwt>`

In [ ]:
# Variables
APP_PASSWORD = ""
BASE_URL = "https://api.bsky.app/xrpc"
HANDLE = "moresandeep"
APPVIEW_BASE = "https://public.api.bsky.app"

# Request
login_url = f"{BASE_URL}/com.atproto.server.createSession"
payload = {"identifier": HANDLE, "password": APP_PASSWORD}
r = requests.post(login_url, json=payload, timeout=30)
displayResponse(r)

Error: 404 <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8">
<title>Error</title>
</head>
<body>
<pre>Cannot POST /xrpc/com.atproto.server.createSession</pre>
</body>
</html>

